In [1]:
import glob,os
import numpy as np
import pandas as pd


import GCRCatalogs as gcr
import healpy as hp

import astropy
from fitsio import FITS
from astropy.table import Table, setdiff, vstack, join_skycoord, join
from astropy.coordinates import SkyCoord
import astropy.units as u

import matplotlib.pyplot as plt
plt.style.use('MNRAS.mplstyle')

file_path = r"/global/u1/k/kamau/SE-CLMM-LSSTDESC/project-1/Data/"

## Load the redMaPPer catalog

In [11]:
redmapper = gcr.load_catalog('skysim5000_v1.1.1_redmapper_v0.8.5')
redmapper_cl = Table(redmapper.get_quantities(['cluster_id','ra','dec','redshift','richness','clusters/z_lambda','clusters/mem_match_id'])) #46491
redmapper_me = Table(redmapper.get_quantities(['cluster_id_member','dec_member','ra_member','p_member'])) #2381320
print(len(redmapper_cl), len(redmapper_me))

46491 2381320


In [12]:
redmapper_cl['redshift'].min(), redmapper_cl['redshift'].max()

(0.08654202, 1.1269356)

In [8]:
redmapper_cl = redmapper_cl[(redmapper_cl['redshift']>=0.2) & (redmapper_cl['redshift']<=1)]
print(len(redmapper_cl))
zranges = [(0.2,0.35), (0.35, 0.5), (0.5, 0.65), (0.65, 0.8), (0.8,1)] #, 
lamranges = [(20,30), (30,45), (45,60), (60,500)]
zlabels = ['(0.2,0.35]', '(0.35, 0.5]', '(0.5, 0.65]', '(0.65, 0.8]', '(0.8,0.95]'] #, '(0.8,0.95]'
lamlabels = ['(20,30]', '(30,45]', '(45,60]', '(60,500]']
redmapper_cl['redshift_range'] = pd.cut(redmapper_cl['redshift'], bins=[r[0] for r in zranges] + [zranges[-1][-1]], labels=zlabels)
redmapper_cl['lambda_range'] = pd.cut(redmapper_cl['richness'], bins=[r[0] for r in lamranges] + [lamranges[-1][-1]], labels=lamlabels)

grouped = redmapper_cl.group_by(['redshift_range', 'lambda_range'])
cluster_counts = grouped.groups.aggregate(len)
cluster_counts

38630


dec,richness,redshift,clusters/z_lambda,ra,cluster_id,clusters/mem_match_id,redshift_range,lambda_range
int64,int64,int64,int64,int64,int64,int64,object,object
2381,2381,2381,2381,2381,2381,2381,"(0.2,0.35]","(20,30]"
1311,1311,1311,1311,1311,1311,1311,"(0.2,0.35]","(30,45]"
523,523,523,523,523,523,523,"(0.2,0.35]","(45,60]"
599,599,599,599,599,599,599,"(0.2,0.35]","(60,500]"
4046,4046,4046,4046,4046,4046,4046,"(0.35, 0.5]","(20,30]"
2182,2182,2182,2182,2182,2182,2182,"(0.35, 0.5]","(30,45]"
854,854,854,854,854,854,854,"(0.35, 0.5]","(45,60]"
886,886,886,886,886,886,886,"(0.35, 0.5]","(60,500]"
5050,5050,5050,5050,5050,5050,5050,"(0.5, 0.65]","(20,30]"


## Load the Halos (GCR)

In [3]:
# ## Sigma and DS profile
sigma_ds_profile1 = pd.read_pickle(file_path + '/WL-Signal/skysim-full-2152757-1.1.1.csv')
sigma_ds_profile1 = sigma_ds_profile1[(sigma_ds_profile1['redshift']>=0.2) & (sigma_ds_profile1['redshift']<=1)]
sigma_ds_profile1 = Table.from_pandas(sigma_ds_profile1)
sigma_ds_profile1['halo_id'] = sigma_ds_profile1['halo_id'].astype(int)
print(len(sigma_ds_profile1))

sigma_ds_profile2 = Table.read(file_path + 'halos/Skysim_SOD_Filter/skysim_0.2-1_1.1.1_SOD.dat', format='ascii')
print(len(sigma_ds_profile2))
print(len(sigma_ds_profile2))

2152757
9132204
9132204


In [6]:
 '{:e}'.format(sigma_ds_profile2['halo_mass'].min())

'2.160281e+11'

In [7]:
redmapper_cl = redmapper_cl[(redmapper_cl['redshift']>=0.2) & (redmapper_cl['redshift']<=1)] #38284
print(len(redmapper_cl))

38630


In [8]:
perfect_match2 = join(redmapper_cl, sigma_ds_profile2, join_type='inner', keys=['dec','ra'])
print(len(perfect_match2))
unmatchedRedCl = redmapper_cl[~np.isin(redmapper_cl['cluster_id'],perfect_match2['cluster_id'])] # 5092
print(len(unmatchedRedCl))
unmatchedhalos = sigma_ds_profile2[~np.isin(sigma_ds_profile2['halo_id'],perfect_match2['halo_id'])]
print(len(unmatchedhalos))

33345
5285
9098859


## Setting tolerance for matching the remaining RedMaPPer clusters with halos

In [9]:
unmatchedhalos['sc'] = SkyCoord(unmatchedhalos['ra'], unmatchedhalos['dec'], unit='deg')
unmatchedRedCl['sc'] = SkyCoord(unmatchedRedCl['ra'], unmatchedRedCl['dec'], unit='deg')

In [10]:
matched_SC = join(unmatchedRedCl, unmatchedhalos, keys='sc', join_funcs={'sc': join_skycoord(0.01 * u.deg)})
print(len(matched_SC))

filtered_table = Table(names=matched_SC.colnames, dtype=matched_SC.dtype)

grouped_table = matched_SC.group_by('sc_id')
for group in grouped_table.groups:
    if len(group) == 1:
        filtered_table.add_row(group[0])
    else:
        redshift_diff = np.abs(group['redshift_1'] - np.median(group['redshift_2']))
        min_diff_idx = np.argmin(redshift_diff)
        filtered_table.add_row(group[min_diff_idx])
        
print(len(np.unique(filtered_table['clusters/mem_match_id']))) # 4998, 79687
filtered_table[:5]

7853
4850


sc_id,ra_1,clusters/z_lambda,cluster_id,clusters/mem_match_id,dec_1,richness,redshift_1,sc_1,halo_id,baseDC2/sod_halo_mass,halo_mass,ra_2,dec_2,redshift_2,baseDC2/sod_halo_radius,pixel_id,sc_2
int64,float64,float32,int32,int32,float64,float32,float32,object,int64,float64,float64,float64,float64,float64,float64,int64,object
1,2.896798269949837,0.5912586,1365,1365,-1.5203787028802902,87.18639,0.5912586,"<SkyCoord (ICRS): (ra, dec) in deg (2.89679827, -1.5203787)>",1175106209307,215036714811392.0,421214361548078.9,2.8949591873192695,-1.5172599429600495,0.5910195826867897,1.2696400880813599,6209,"<SkyCoord (ICRS): (ra, dec) in deg (2.89495919, -1.51725994)>"
2,3.9520044556241634,0.5922861,6286,6286,-1.2222302793974655,53.274582,0.5922861,"<SkyCoord (ICRS): (ra, dec) in deg (3.95200446, -1.22223028)>",30606209307,309910478782464.0,414362641053205.6,3.9539879408698955,-1.229060036928648,0.5917466111825231,1.4341225624084473,6209,"<SkyCoord (ICRS): (ra, dec) in deg (3.95398794, -1.22906004)>"
3,7.237337018664522,0.73358476,7309,7309,-1.7915726702320818,51.672813,0.73358476,"<SkyCoord (ICRS): (ra, dec) in deg (7.23733702, -1.79157267)>",503406338286,137055065604096.0,224705642496000.0,7.233703647140288,-1.789699135266021,0.724829526271753,1.12343430519104,6338,"<SkyCoord (ICRS): (ra, dec) in deg (7.23370365, -1.78969914)>"
4,2.131558544832236,0.24652696,9909,9909,-0.7840742485998262,43.45038,0.24652696,"<SkyCoord (ICRS): (ra, dec) in deg (2.13155854, -0.78407425)>",3127906209259,8907119394816.0,12113564213363.38,2.125421402647639,-0.7769044372884384,0.9015389790334394,0.4663834869861603,6209,"<SkyCoord (ICRS): (ra, dec) in deg (2.1254214, -0.77690444)>"
5,7.622600010501734,0.84468967,8472,8472,-1.1842702769604658,52.877953,0.84468967,"<SkyCoord (ICRS): (ra, dec) in deg (7.62260001, -1.18427028)>",2566206211266,208761079726080.0,269065074197633.8,7.624281569585428,-1.1864275777400628,0.8589438541799852,1.325057864189148,6211,"<SkyCoord (ICRS): (ra, dec) in deg (7.62428157, -1.18642758)>"


In [92]:
# filtered_table['redshift_1','dec_1','ra_1','ra_2','dec_2','redshift_2']

In [40]:
filtered_table.remove_columns(['ra_1','dec_1','sc_id','sc_1','sc_2'])
filtered_table.rename_columns(['ra_2','dec_2','redshift_2'],['ra','dec','redshift'])

In [41]:
cluster_halo = vstack([perfect_match2, filtered_table])
print(len(cluster_halo))

38195


In [42]:
# cluster_halo.write(file_path + 'halos/halos_redmapper_SOD_0.2-1_SOD.dat', format='ascii')

In [43]:
print(len(cluster_halo[(cluster_halo['redshift_2']>=0.2) & (cluster_halo['redshift_2']<=0.35)]))

4465


# Method 2: Cluster-Halo Matching
## Step 1: 
Perfect match based on the cluster and halo central coordinates which perfectly match (centered)
## Step 2:
<ol>
    <li> Search for halo-cluster pairs with redshift separation Δ𝑧 ≤ 0.05 between cluster photometric redshift and true halo redshift. </li>
    <li>  For each halo, identify those redMaPPer clusters with BCGs within a projected 2-D, comoving radius of 2ℎ−1 Mpc of
the halo central galaxy. I used angular separation. </li> 
    <li> If there are multiple redMaPPer clusters satisfying these separation criteria, we match the halo to the richest cluster that hasn’t been previously matched.</li> 
    <li> For each cluster, we repeat this matching process, selecting halos satisfying the redshift and projected distance criteria, and then choosing the most massive such halo still on the list as the one to be associated with that cluster. </li> 
    <li> Clusters and halos that uniquely match with each other in both matching steps are considered valid matches.</li> 
</ol>

In [98]:
# import astropy.units as u
# from astropy.coordinates import SkyCoord
# from astropy.cosmology import FlatLambdaCDM
# skysim = gcr.load_catalog('skysim5000_v1.1.1')
# skycosmology = skysim.cosmology
# cosmoskysim = FlatLambdaCDM(H0=skycosmology.H0.value, Om0= skycosmology.Om0)

# def calculate_physicaldist_separation(ha_z, cl_z):
#     cluster_coords = cosmoskysim.comoving_distance(cl_z) / (1+ cl_z)
#     halo_coords = cosmoskysim.comoving_distance(ha_z) / (1+ ha_z)
#     separation_cl = np.abs(cluster_coords - halo_coords)
#     separation_halo = np.abs(halo_coords - cluster_coords)
#     return separation_cl, separation_halo

# calculate_physicaldist_separation(ha_z=0.4248974370455487, cl_z = 0.43412662)[0],calculate_physicaldist_separation(cl_z=0.43412662,ha_z=0.4248974370455487)[1]

In [14]:
import astropy.units as u
from astropy.coordinates import SkyCoord

def calculate_angular_separation(cl_ra, cl_dec, ha_ra2, ha_dec):
    cluster_coords = SkyCoord(ra=cl_ra, dec=cl_dec, unit='deg')
    halo_coords = SkyCoord(ra=ha_ra2, dec=ha_dec, unit='deg')
    separation = cluster_coords.separation(halo_coords)
    return separation
calculate_angular_separation(cl_ra=2.896798269949837,cl_dec=-1.5203787028802902,ha_ra2=2.8949591873192695,ha_dec=-1.5172599429600495)

<Angle 0.00362029 deg>

In [19]:

col_names = ['halo_id', 'baseDC2/sod_halo_mass', 'halo_mass', 'ra', 'dec', 'redshift_2', 'baseDC2/sod_halo_radius', 'pixel_id',
            'clusters/mem_match_id','dec_cl','redshift_1','ra_cl','cluster_id','richness','clusters/z_lambda']

data_types = ['<i8','<f8','<f8','<f8','<f8','<f8','<f8','<i8','<i4','<f8','<f4','<f8','<i4','<f4','<f4']

In [20]:
from astropy.table import join, Table
from astropy.coordinates import SkyCoord
from astropy import units as u

def match_clusters_to_halos(redMaPPer_clusters, Halos):
    # Perform a perfect join based on 'ra' and 'dec'
    centered_clusters = join(redMaPPer_clusters, Halos, keys=['ra','dec'], table_names=['redMaPPer', 'Halos'])
    
    print(len(centered_clusters)) #, len(centered_clusters)
    
    # Match miscentered clusters
    unmatched_halos = Halos[~np.isin(Halos['halo_id'],centered_clusters['halo_id'])]
    # print("Number of unmatched halos:", len(unmatched_halos))
    sorted_indices_halos = unmatched_halos['baseDC2/sod_halo_mass'].argsort()[::-1]
    
    miscentered_clusters = redMaPPer_clusters[~np.isin(redMaPPer_clusters['cluster_id'],centered_clusters['cluster_id'])]
    # print(len(miscentered_clusters))
    sorted_indices_cl = miscentered_clusters['richness'].argsort()[::-1]
    
    
    halo_clusters = []
    
    for index in sorted_indices_halos:
        halo = unmatched_halos[index]
        # Search for halos within redshift separation and projected distance range
        delta_z = (abs(halo['redshift'] - miscentered_clusters['redshift']) <= 0.05)
        separation = (calculate_angular_separation(cl_ra=miscentered_clusters['ra'], cl_dec=miscentered_clusters['dec'],
                                                   ha_ra2=halo['ra'], ha_dec=halo['dec']) <= 0.01 * u.deg)
        candidate_clusters = miscentered_clusters[delta_z & separation]
        
        halo_clusters = []
        if len(candidate_clusters)>0:
            matched_clusters = candidate_clusters[candidate_clusters['richness'].argmax()]
            # print(matched_clusters)
            # print(halo)
            matched_clusters = np.array(list(matched_clusters))
            halo = np.array(list(halo))
            halo_clusters.append(np.hstack((halo,matched_clusters)))
        

    for index in sorted_indices_cl:
        cluster = miscentered_clusters[index]
        # Search for halos within redshift separation and projected distance range
        delta_z = (abs(cluster['redshift'] - unmatched_halos['redshift']) <= 0.05)
        separation = (calculate_angular_separation(cl_ra=cluster['ra'], cl_dec=cluster['dec'],
                                                   ha_ra2=unmatched_halos['ra'], ha_dec=unmatched_halos['dec']) <= 0.01 * u.deg)
        candidate_halos = unmatched_halos[delta_z & separation]
        
        if len(candidate_halos)>0:
            matched_halos = candidate_halos[candidate_halos['baseDC2/sod_halo_mass'].argmax()]
            # print(cluster['ra','dec','redshift'])
            # print(matched_halos['ra','dec','redshift'])
            matched_halos = np.array(list(matched_halos))
            cluster = np.array(list(cluster))
            halo_clusters.append(np.hstack((matched_halos, cluster)))
            
    # print(halo_clusters)
    
    matching_2 = Table(rows=halo_clusters, names=col_names, dtype=data_types)
        
    print(len(matching_2))
    
    return matching_2  #4579


In [21]:
# match_clusters_to_halos(redmapper_cl, sigma_ds_profile2)

In [22]:
matching_2 = match_clusters_to_halos(redmapper_cl, sigma_ds_profile2)

33345
4579


In [ ]:
# centered = match_clusters_to_halos(redmapper_cl, sigma_ds_profile2)[0]

In [26]:
matching_2[:5]

halo_id,baseDC2/sod_halo_mass,halo_mass,ra,dec,redshift,baseDC2/sod_halo_radius,pixel_id,redshift_cl,ra_cl,dec_cl,clusters/mem_match_id,clusters/z_lambda,cluster_id,richness
int64,float64,float64,float64,float64,float64,float64,int64,float32,float64,float64,int32,float32,int32,float32
387810675347,946464561823744.0,1659895028213904.2,36.64702595786081,-48.635476906871524,0.40586065171152885,1.9710586071014404,10675,0.40834615,36.651313882479684,-48.635005704808485,8,0.40834615,8,283.30435
152507764373,1099301174378496.0,1876985964965138.0,56.2051982276388,-15.572899560512406,0.31597800409397614,1.9939948320388794,7764,0.31853953,56.20271992052518,-15.563425078600355,18,0.31853953,18,268.15616
1929211529307,632555132944384.0,900082137197611.4,7.786037584145139,-60.874670498078295,0.595378021938449,1.8191717863082886,11449,0.60669804,7.7897354641804695,-60.88114179393029,37,0.60669804,37,259.3693
1077508897331,606560044711936.0,963326664865532.5,4.073002035134848,-26.51228821876606,0.47609839213312743,1.7393486499786377,8897,0.48064712,4.068888662939389,-26.514320998633462,29,0.48064712,29,259.0337
901711205323,368588422643712.0,565778905843740.9,84.73005476321971,-56.04144684539624,0.5340020246395574,1.4873884916305542,11205,0.54247814,84.7340498409233,-56.04696471682986,41,0.54247814,41,220.66742


In [37]:
# matching_2.write(file_path + 'halos/halos_redmapper_matching_2.dat', format='ascii')
matching_2 = Table.read(file_path + 'halos/halos_redmapper_matching_2.dat', format='ascii')

In [43]:
matching_2.remove_columns(['ra_cl','dec_cl',])
matching_2.rename_columns(['redshift_2'],['redshift'])
perfect_match2.rename_columns(['redshift_2'],['redshift'])

In [49]:
sigma_ds_profile12 = sigma_ds_profile1['halo_id','radius','sigma','DS','baseDC2/sod_halo_mass','redshift','ra','dec']
print(sigma_ds_profile1.columns)

<TableColumns names=('halo_id','radius','sigma','DS','baseDC2/sod_halo_mass','baseDC2/sod_halo_radius','redshift','dec','magnification','hostHaloMass','ra','halo_mass','pixel_id')>


In [56]:
cluster_halo22 = join(cluster_halo2,sigma_ds_profile12, keys=['halo_id','baseDC2/sod_halo_mass','redshift','ra','dec'],join_type='inner')

In [11]:
# cluster_halo22 = cluster_halo22.to_pandas()
# cluster_halo22.to_pickle(file_path + 'halos/halos_redmapper_mthd2.csv')
halos_redmapper = pd.read_pickle(file_path + 'halos/halos_redmapper_mthd2.csv')
print(len(halos_redmapper))
halos_redmapper = halos_redmapper.drop(columns=['radius','sigma','DS'])
halos_redmapper[:3]

37798


,cluster_id,richness,dec,redshift_1,clusters/z_lambda,clusters/mem_match_id,ra,halo_id,baseDC2/sod_halo_mass,halo_mass,redshift,baseDC2/sod_halo_radius,pixel_id
0,9433,49.796448,-16.994633,0.277185,0.277185,9433,30.689553,7882382,9.793396e+13,2.120835e+14,0.269174,0.877747,7882
1,7477,45.357254,-23.195630,0.942758,0.942758,7477,38.216131,8526253,4.423530e+14,5.598655e+14,0.946937,1.713573,8526
2,21789,27.310617,-30.513351,0.965913,0.965913,21789,39.685389,9294247,7.379789e+13,9.823534e+13,0.970221,0.956616,9294


In [12]:
## New computed DS and S which has h in the distances.
sigma_ds_profile = pd.read_pickle(file_path + 'WL-Signal/skysim-full-DS_S-2152757.csv')
print(len(sigma_ds_profile))
sigma_ds_profile = sigma_ds_profile[['halo_id', 'comoving_dis','Angular_dis','radius','sigma','DS']]
sigma_ds_profile[:3]

2152757


,halo_id,comoving_dis,Angular_dis,radius,sigma,DS
0,21510048411,578.542239,481.558077,"[0.1308730625060924, 0.20814712387717377, 0.33...","[115148363120795.56, 51872210454439.24, 229531...","[-130550254502468.84, -76217790266760.5, -3994..."
1,25010048411,581.180475,483.367013,"[0.13261201512396317, 0.20923342305629689, 0.3...","[7445362739885.1875, 8543016543686.46, -375534...","[-51159678940917.43, -21245333215964.383, -223..."
2,25410048411,578.216572,481.334566,"[0.13437702148667455, 0.21206787125180884, 0.3...","[98287762786790.84, 51371879042919.66, -144157...","[-171453712604657.56, -101776907593780.67, -73..."


In [13]:
merged_cl_halos = halos_redmapper.merge(sigma_ds_profile, on='halo_id', how='left')
print(len(merged_cl_halos))
merged_cl_halos.reset_index(drop=True)
merged_cl_halos[:3]

37798


,cluster_id,richness,dec,redshift_1,clusters/z_lambda,clusters/mem_match_id,ra,halo_id,baseDC2/sod_halo_mass,halo_mass,redshift,baseDC2/sod_halo_radius,pixel_id,comoving_dis,Angular_dis,radius,sigma,DS
0,9433,49.796448,-16.994633,0.277185,0.277185,9433,30.689553,7882382,9.793396e+13,2.120835e+14,0.269174,0.877747,7882,761.430165,599.941336,"[0.13077678865824494, 0.21217091891370196, 0.3...","[110727389393014.08, 58813523100774.29, 102856...","[-8832006208314.781, -23267165687616.883, -127..."
1,7477,45.357254,-23.195630,0.942758,0.942758,7477,38.216131,8526253,4.423530e+14,5.598655e+14,0.946937,1.713573,8526,2267.503546,1164.651511,"[0.13646283273225385, 0.19977708240986783, 0.3...","[2249574075879.025, -3635266351607.388, 282590...","[17803158959296.477, 52395860504862.1, 8447713..."
2,21789,27.310617,-30.513351,0.965913,0.965913,21789,39.685389,9294247,7.379789e+13,9.823534e+13,0.970221,0.956616,9294,2309.790961,1172.351381,"[0.14539961083596725, 0.19753496022432523, 0.3...","[26065599372806.51, 47601960575787.414, -22009...","[53660393591007.734, 16341062563337.203, -1195..."


In [16]:
# merged_cl_halos.to_pickle(file_path + 'halos/halos_redmapper_withh_mthd2.csv')